In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#change dir as required
root_dir = '/content/drive/MyDrive/Kaggle/pred-gene-exp'


### Define train dataset and split train-valid

In [ ]:
def read_fasta_as_string(fasta_filename):
    """
    Reads a FASTA file and returns the sequence as a string.

    Parameters:
    - fasta_filename (str): The name of the FASTA file to read.

    Returns:
    - str: The DNA sequence as a string.
    """
    sequence = ""
    with open(fasta_filename, 'r') as fasta_file:
        for line in fasta_file:
            # Skip the header line
            if line.startswith(">"):
                continue
            # Remove newline characters and concatenate
            sequence += line.strip()
    return sequence

In [ ]:
!pip install kipoiseq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 95.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 23.2.0
    Uninstalling attrs-23.2.0:
      Succes

In [ ]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from kipoiseq.transforms.functional import one_hot_dna

class ExpDataset(Dataset):
    """Dataset for reading fasta and exp."""

    def __init__(self, csv_file, fasta_dir='./fasta'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
        """
        # Read the CSV file
        self.data_frame = pd.read_csv(csv_file)
        self.fasta_dir = fasta_dir
        if self.data_frame.shape[1] == 1:
            self.has_target = False
        else:
            self.has_target = True

    def __len__(self):
        # Return the number of rows in the DataFrame
        return len(self.data_frame)

    def __getitem__(self, idx):
        sample = {}
        rec = self.data_frame.iloc[idx]
        seq_filename = self.fasta_dir + '/' + rec['SeqId'] + '.fa'
        sample['seq'] = torch.tensor(
            one_hot_dna(
                read_fasta_as_string(
                    seq_filename)).astype(np.float32))
        if self.has_target:
            sample['target'] = torch.tensor(
                rec.iloc[1:].values.astype(np.float32))
        return sample

In [ ]:
train_ds = ExpDataset(root_dir +'/hackathon_train.csv', root_dir + '/fasta')

In [ ]:
#View size
dataset_size = len(train_ds)
print(f"Dataset size: {dataset_size}")

Dataset size: 14864


In [ ]:
from torch.utils.data import random_split

dataset_size = len(train_ds)
valid_size = int(dataset_size * 0.2)  # 20% for validation
train_size = dataset_size - valid_size  # Remaining for training

# Split the dataset
train_ds, valid_ds = random_split(train_ds, [train_size, valid_size])

In [ ]:
len(train_ds), len(valid_ds)

(11892, 2972)

In [ ]:
from torch.utils.data import DataLoader
# Define the DataLoader
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
valid_dl = DataLoader(valid_ds, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
len(train_dl), len(valid_dl)

(186, 47)

### Baseline model training

In [ ]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {device}")

Training on cuda


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

def calculate_output_size(input_size, kernel_size, stride):
    return (input_size - kernel_size) // stride + 1

class GeneExpressionBranch(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, pool_stride, fc_size, sequence_length, output_size, dropout_rate=0.5):
        super(GeneExpressionBranch, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size)
        self.pool = nn.AvgPool1d(kernel_size=kernel_size, stride=pool_stride)
        self.dropout = nn.Dropout(dropout_rate)
        self.batch_norm = nn.BatchNorm1d(out_channels)

        # Accurate output size calculation
        conv_output_size = calculate_output_size(sequence_length, kernel_size, 1)
        pool_output_size = calculate_output_size(conv_output_size, kernel_size, pool_stride)

        self.fc1 = nn.Linear(out_channels * pool_output_size, fc_size)
        self.fc2 = nn.Linear(fc_size, output_size)

        # Weight initialization
        nn.init.kaiming_normal_(self.conv1.weight, mode='fan_out', nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc1.weight, mode='fan_out', nonlinearity='relu')
        nn.init.kaiming_normal_(self.fc2.weight, mode='fan_out', nonlinearity='relu')

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Rearrange dimensions to [batch, channels, sequence_length]
        x = F.relu(self.conv1(x))
        x = self.batch_norm(x)
        x = self.pool(x)
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

class GeneExpressionEnsemble(nn.Module):
    def __init__(self, sequence_length=49152, output_size=399):
        super(GeneExpressionEnsemble, self).__init__()
        self.branch1 = GeneExpressionBranch(4, 128, 10, 900, 640, sequence_length, output_size)
        self.branch2 = GeneExpressionBranch(4, 64, 15, 450, 704, sequence_length, output_size)
        self.branch3 = GeneExpressionBranch(4, 128, 20, 300, 2048, sequence_length, output_size)

    def forward(self, x):
        out1 = self.branch1(x)
        out2 = self.branch2(x)
        out3 = self.branch3(x)
        combined = torch.mean(torch.stack([out1, out2, out3]), dim=0)
        return combined


In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), root_dir + '/hackathon_model_final_basemodel_earlystop.pth')
        self.val_loss_min = val_loss


In [ ]:
from torch.optim.lr_scheduler import StepLR

model = GeneExpressionEnsemble()
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)
valid_dl = DataLoader(valid_ds, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)  # L2 regularization
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GeneExpressionEnsemble(
  (branch1): GeneExpressionBranch(
    (conv1): Conv1d(4, 128, kernel_size=(10,), stride=(1,))
    (pool): AvgPool1d(kernel_size=(10,), stride=(900,), padding=(0,))
    (dropout): Dropout(p=0.5, inplace=False)
    (batch_norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc1): Linear(in_features=7040, out_features=640, bias=True)
    (fc2): Linear(in_features=640, out_features=399, bias=True)
  )
  (branch2): GeneExpressionBranch(
    (conv1): Conv1d(4, 64, kernel_size=(15,), stride=(1,))
    (pool): AvgPool1d(kernel_size=(15,), stride=(450,), padding=(0,))
    (dropout): Dropout(p=0.5, inplace=False)
    (batch_norm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc1): Linear(in_features=7040, out_features=704, bias=True)
    (fc2): Linear(in_features=704, out_features=399, bias=True)
  )
  (branch3): GeneExpressionBranch(
    (conv1): Conv1d(4, 128, kernel_size=(20,), stride=(1,))

**define model metric**

In [ ]:
def r_squared(y_true, y_pred):
    # Calculate the total sum of squares (SST)
    sst = torch.sum((y_true - torch.mean(y_true)) ** 2)
    # Calculate the residual sum of squares (SSR)
    ssr = torch.sum((y_true - y_pred) ** 2)
    # Calculate the R^2 score
    r2 = 1 - ssr / (sst + 1e-8)
    return r2

**define train and validate routines**

In [ ]:
# Define root directory where models should be saved
root_dir = '/content/drive/MyDrive/Kaggle/pred-gene-exp'


In [ ]:
import torch
from tqdm.notebook import tqdm
from torch.optim.lr_scheduler import StepLR

def reset_weights(m):
    if hasattr(m, 'reset_parameters'):
        print(f'Resetting parameters of layer = {m.__class__.__name__}')
        m.reset_parameters()

def train_model(model, train_dl, valid_dl, criterion, optimizer, scheduler, device, num_epochs, root_dir):
    best_metric = float('-inf')
    early_stopping = EarlyStopping(patience=10, verbose=True)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_metric = 0.0
        for batch in tqdm(train_dl):
            inputs, targets = batch['seq'], batch['target']
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            metric = r_squared(targets, outputs)
            running_metric += metric * inputs.size(0)

        epoch_loss = running_loss / len(train_dl.dataset)
        epoch_metric = running_metric / len(train_dl.dataset)
        print(f'Epoch {epoch+1}, Loss: {epoch_loss:.4f}, R^2: {epoch_metric:.4f}')

        val_loss, val_metric = validate_model(valid_dl, model, criterion, device, early_stopping)
        scheduler.step()

        if early_stopping.early_stop:
            print("Early stopping triggered")
            break

    # Load the best model
    model.load_state_dict(torch.load(root_dir + '/hackathon_model_final_basemodel_earlystop.pth'))
    return epoch_loss, epoch_metric

def validate_model(dl, model, criterion, device, early_stopping):
    model.eval()
    val_loss = 0.0
    val_metric = 0.0
    with torch.no_grad():
        for batch in dl:
            inputs, targets = batch['seq'], batch['target']
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
            metric = r_squared(targets, outputs)
            val_metric += metric * inputs.size(0)

    val_loss /= len(dl.dataset)
    val_metric /= len(dl.dataset)
    print(f'Validation Loss: {val_loss:.4f}, Validation R^2: {val_metric:.4f}')

    # Call early stopping
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        return val_loss, val_metric  # If early stopping, return immediately

    return val_loss, val_metric

# Initialize model weights
model.apply(reset_weights)

# Train the model
num_epochs = 10
loss, r_squared_value = train_model(
    model, train_dl, valid_dl, criterion, optimizer,
    StepLR(optimizer, step_size=10, gamma=0.1), device, num_epochs, root_dir)

Resetting parameters of layer = Conv1d
Resetting parameters of layer = BatchNorm1d
Resetting parameters of layer = Linear
Resetting parameters of layer = Linear
Resetting parameters of layer = Conv1d
Resetting parameters of layer = BatchNorm1d
Resetting parameters of layer = Linear
Resetting parameters of layer = Linear
Resetting parameters of layer = Conv1d
Resetting parameters of layer = BatchNorm1d
Resetting parameters of layer = Linear
Resetting parameters of layer = Linear


  0%|          | 0/186 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1, Loss: 5.5914, R^2: -0.0457
Validation Loss: 5.2201, Validation R^2: 0.0618
Validation loss decreased (inf --> 5.220072).  Saving model ...


  0%|          | 0/186 [00:00<?, ?it/s]

...

Epoch 8, Loss: 4.0413, R^2: 0.2429
Validation Loss: 4.7783, Validation R^2: 0.1423
Validation loss decreased (4.792220 --> 4.778273).  Saving model ...


  0%|          | 0/186 [00:00<?, ?it/s]

Epoch 9, Loss: 3.9860, R^2: 0.2542
Validation Loss: 4.7804, Validation R^2: 0.1419
EarlyStopping counter: 1 out of 10


  0%|          | 0/186 [00:00<?, ?it/s]

Epoch 10, Loss: 3.9573, R^2: 0.2586
Validation Loss: 4.7268, Validation R^2: 0.1506
Validation loss decreased (4.778273 --> 4.726772).  Saving model ...


### Make prediction on the test set for submission

In [ ]:
from tqdm.notebook import tqdm

# Function to predict using the model
def predict_model(dl):
    model.eval()  # Set the model to evaluation mode
    out_pool = []
    with torch.no_grad():  # No gradient computation
        for batch in tqdm(dl):
            inputs = batch['seq']
            inputs = inputs.to(device)  # Move data to GPU
            outputs = model(inputs)
            out_pool.append(outputs.cpu())

    return torch.cat(out_pool)

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Kaggle/pred-gene-exp/hackathon_model_final_basemodel_earlystop.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
test_masked_ds = ExpDataset(root_dir +'/hackathon_test_masked.csv', root_dir + '/fasta')

In [ ]:
test_masked_dl = DataLoader(test_masked_ds, batch_size=64, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
test_out = predict_model(test_masked_dl)

  0%|          | 0/79 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
test_out.shape

torch.Size([5000, 399])

In [ ]:
test_out_df = pd.DataFrame(test_out.numpy(), columns=train_ds.dataset.data_frame.columns[1:])

In [ ]:
pred_df = pd.concat([test_masked_ds.data_frame, test_out_df], axis=1)

In [ ]:
pred_df.head()

,SeqId,E0001,E0002,E0003,E0004,E0005,E0006,E0007,E0008,E0009,...,E0390,E0391,E0392,E0393,E0394,E0395,E0396,E0397,E0398,E0399
0,S010036,3.170789,2.973635,2.814799,2.874613,2.862776,2.994923,2.899171,2.974175,2.841834,...,3.058355,2.859570,3.194716,3.359787,2.984541,3.118995,3.226507,3.187487,3.275448,3.235953
1,S012718,3.605296,3.417747,3.192517,3.312485,3.277514,3.379172,3.300030,3.384032,3.247930,...,3.578639,3.020983,3.621942,3.770847,3.226720,3.382427,3.770030,3.524527,3.697136,3.570424
2,S013672,2.260146,2.160052,2.011872,2.101192,2.068870,2.119050,2.076772,2.132212,2.051342,...,2.309303,1.799379,2.279266,2.360426,1.980410,2.059000,2.416129,2.159085,2.310679,2.189067
3,S011941,2.288809,2.185049,2.045996,2.113737,2.104731,2.179042,2.121662,2.171275,2.081859,...,2.207018,2.035707,2.297903,2.389376,2.092022,2.214642,2.343796,2.318986,2.356895,2.319290
4,S000275,1.187978,1.034119,1.031406,0.993177,0.991679,1.099508,1.031601,1.074066,1.003025,...,1.075296,1.232215,1.207668,1.330463,1.281460,1.294594,1.124865,1.221852,1.247038,1.295953


In [ ]:
pred_df.to_csv('Hackathon_Model3.csv', index=False)

In [ ]:
from google.colab import files
files.download('Hackathon_Model3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>